In [30]:
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from data import BodyPart
import tensorflow as tf

In [31]:
def load_csv(csv_path):
    df = pd.read_csv(csv_path)
    df.drop(['filename'],axis=1, inplace=True)
    classes = df.pop('class_name').unique()
    y = df.pop('class_no')
    
    X = df.astype('float64')
    y = keras.utils.to_categorical(y)
    
    return X, y, classes

In [32]:
def preprocess_data(X_train):
    processed_X_train = []
    for i in range(X_train.shape[0]):
        embedding = landmarks_to_embedding(tf.reshape(tf.convert_to_tensor(X_train.iloc[i]), (1, 51)))
        processed_X_train.append(tf.reshape(embedding, (34)))
    return tf.convert_to_tensor(processed_X_train)


In [33]:

def landmarks_to_embedding(landmarks_and_scores):
    """Converts the input landmarks into a pose embedding."""
    # Reshape the flat input into a matrix with shape=(17, 3)
    reshaped_inputs = keras.layers.Reshape((17, 3))(landmarks_and_scores)

    # Normalize landmarks 2D
    landmarks = normalize_pose_landmarks(reshaped_inputs[:, :, :2])
    # Flatten the normalized landmark coordinates into a vector
    embedding = keras.layers.Flatten()(landmarks)
    return embedding


In [34]:

def normalize_pose_landmarks(landmarks):
    """Normalizes the landmarks translation by moving the pose center to (0,0) and
    scaling it to a constant pose size.
  """
  # Move landmarks so that the pose center becomes (0,0)
    pose_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    pose_center = tf.expand_dims(pose_center, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to perform
    # substraction
    pose_center = tf.broadcast_to(pose_center, 
                                [tf.size(landmarks) // (17*2), 17, 2])
    landmarks = landmarks - pose_center

    # Scale the landmarks to a constant pose size
    pose_size = get_pose_size(landmarks)
    landmarks /= pose_size
    return landmarks

In [35]:
def get_center_point(landmarks, left_bodypart, right_bodypart):
    """Calculates the center point of the two given landmarks."""
    left = tf.gather(landmarks, left_bodypart.value, axis=1)
    right = tf.gather(landmarks, right_bodypart.value, axis=1)
    center = left * 0.5 + right * 0.5
    return center

In [36]:
def get_pose_size(landmarks, torso_size_multiplier=2.5):
    """Calculates pose size.

    It is the maximum of two values:
    * Torso size multiplied by `torso_size_multiplier`
    * Maximum distance from pose center to any pose landmark
    """
    # Hips center
    hips_center = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                 BodyPart.RIGHT_HIP)

    # Shoulders center
    shoulders_center = get_center_point(landmarks, BodyPart.LEFT_SHOULDER,
                                      BodyPart.RIGHT_SHOULDER)

    # Torso size as the minimum body size
    torso_size = tf.linalg.norm(shoulders_center - hips_center)
    # Pose center
    pose_center_new = get_center_point(landmarks, BodyPart.LEFT_HIP, 
                                     BodyPart.RIGHT_HIP)
    pose_center_new = tf.expand_dims(pose_center_new, axis=1)
    # Broadcast the pose center to the same size as the landmark vector to
    # perform substraction
    pose_center_new = tf.broadcast_to(pose_center_new,
                                    [tf.size(landmarks) // (17*2), 17, 2])

    # Dist to pose center
    d = tf.gather(landmarks - pose_center_new, 0, axis=0,
                name="dist_to_pose_center")
    # Max dist to pose center
    max_dist = tf.reduce_max(tf.linalg.norm(d, axis=0))

    # Normalize scale
    pose_size = tf.maximum(torso_size * torso_size_multiplier, max_dist)
    return pose_size

In [37]:
def cnn_model(input_shape, num_classes):
    model = keras.Sequential([
        keras.layers.Conv1D(64, 3, activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling1D(2),
        keras.layers.Conv1D(128, 3, activation='relu'),
        keras.layers.MaxPooling1D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [38]:
X, y, class_names = load_csv('train_data.csv')
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)
X_test, y_test, _ = load_csv('test_data.csv')

In [39]:
processed_X_train = preprocess_data(X_train)
processed_X_val =  preprocess_data(X_val)
processed_X_test = preprocess_data(X_test)

In [40]:
# Reshape data for CNN
processed_X_train = tf.expand_dims(processed_X_train, axis=-1)
processed_X_val = tf.expand_dims(processed_X_val, axis=-1)
processed_X_test = tf.expand_dims(processed_X_test, axis=-1)

In [41]:
input_shape = processed_X_train.shape[1:]

In [42]:
model = cnn_model(input_shape, len(class_names))

C:\Users\Ugyen Dendup\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [43]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [44]:
history = model.fit(processed_X_train, y_train,
                    epochs=200,
                    batch_size=16,
                    validation_data=(processed_X_val, y_val))

Epoch 1/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3586 - loss: 1.6838 - val_accuracy: 0.7533 - val_loss: 0.8708
Epoch 2/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6526 - loss: 0.9640 - val_accuracy: 0.8194 - val_loss: 0.5695
Epoch 3/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7740 - loss: 0.6380 - val_accuracy: 0.9251 - val_loss: 0.2871
Epoch 4/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8812 - loss: 0.3751 - val_accuracy: 0.9515 - val_loss: 0.1811
Epoch 5/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9325 - loss: 0.2309 - val_accuracy: 0.9559 - val_loss: 0.1211
Epoch 6/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9392 - loss: 0.1795 - val_accuracy: 0.9824 - val_loss: 0.0865
Epoch 7/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9599 - loss: 0.1300 - val_accuracy: 0.9736 - val_loss: 0.0679
Epoch 8/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9615 - loss: 0.1159 - val_accuracy: 0.9780 - v

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9979 - loss: 0.0069 - val_accuracy: 0.9912 - val_loss: 0.0260
Epoch 52/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9983 - loss: 0.0051 - val_accuracy: 0.9912 - val_loss: 0.0243
Epoch 53/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9989 - loss: 0.0038 - val_accuracy: 0.9912 - val_loss: 0.0188
Epoch 54/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 0.9912 - val_loss: 0.0253
Epoch 55/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9912 - val_loss: 0.0234
Epoch 56/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 0.9912 - val_loss: 0.0345
Epoch 57/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9999 - loss: 0.0015 - val_accuracy: 0.9956 - val_loss: 0.0234
Epoch 58/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9919 - loss: 0.0260 - val_accuracy: 0.9912 - val_lo

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9997 - loss: 0.0023 - val_accuracy: 0.9956 - val_loss: 0.0259
Epoch 102/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.9912 - val_loss: 0.0192
Epoch 103/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 3.8939e-04 - val_accuracy: 0.9912 - val_loss: 0.0460
Epoch 104/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9912 - val_loss: 0.0246
Epoch 105/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 2.2814e-04 - val_accuracy: 0.9912 - val_loss: 0.0301
Epoch 106/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9995 - loss: 9.0515e-04 - val_accuracy: 0.9868 - val_loss: 0.0589
Epoch 107/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.9956 - val_loss: 0.0163
Epoch 108/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 3.9085e-04 - val_ac

Epoch 151/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 3.3089e-04 - val_accuracy: 0.9956 - val_loss: 0.0205
Epoch 152/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 3.4038e-04 - val_accuracy: 0.9956 - val_loss: 0.0133
Epoch 153/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 1.8943e-04 - val_accuracy: 0.9956 - val_loss: 0.0060
Epoch 154/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9967 - loss: 0.0029 - val_accuracy: 0.9912 - val_loss: 0.0583
Epoch 155/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9968 - loss: 0.0112 - val_accuracy: 1.0000 - val_loss: 0.0023
Epoch 156/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9966 - loss: 0.0084 - val_accuracy: 0.9956 - val_loss: 0.0281
Epoch 157/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9974 - loss: 0.0045 - val_accuracy: 1.0000 - val_loss: 6.0821e-04
Epoch 158/200
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9993 - loss: 0

81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 5.0525e-04 - val_accuracy: 1.0000 - val_loss: 9.6182e-04


In [45]:
print('-----------------EVALUATION----------------')
loss, accuracy = model.evaluate(processed_X_test, y_test)
print('LOSS: ', loss)
print("Test ACCURACY: ", accuracy)

-----------------EVALUATION----------------
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9993 - loss: 0.0051    
LOSS:  0.03279018774628639
Test ACCURACY:  0.9954389929771423


In [46]:
loss, accuracy = model.evaluate(processed_X_train, y_train)
print('LOSS: ', loss)
print("Train ACCURACY: ", accuracy)

41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.4296e-06
LOSS:  1.4047443528397707e-06
Train ACCURACY:  1.0


In [47]:
loss, accuracy = model.evaluate(processed_X_val, y_val)
print('LOSS: ', loss)
print("Train ACCURACY: ", accuracy)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 8.1096e-04 
LOSS:  0.0009618236217647791
Train ACCURACY:  1.0


In [48]:
model.save('CNN_model.keras')

## Model Prection test

In [24]:
# Preprocessor for a single image

import tensorflow as tf
import numpy as np
from movenet import Movenet
import os

# Load MoveNet model
movenet = Movenet('movenet_thunder.tflite')

def detect_landmarks(image, detection_threshold=0.1):
    # Detect landmarks in the given image
    person = movenet.detect(image)
    
    # Check if any person is detected
    if person is None:
        return None
    
    # Check if all landmarks are above the detection threshold
    min_landmark_score = min([keypoint.score for keypoint in person.keypoints])
    if min_landmark_score < detection_threshold:
        return None
    
    # Get landmarks and scale them to the same size as the input image
    pose_landmarks = np.array([[keypoint.coordinate.x, keypoint.coordinate.y, keypoint.score] for keypoint in person.keypoints], dtype=np.float32)
    
    return pose_landmarks

def preprocess_image(image_path, target_shape):
    # Read image
    image = tf.io.read_file(image_path)
    image = tf.io.decode_jpeg(image)
    
    # Resize image to target shape
    image = tf.image.resize(image, target_shape)
    
    # Convert image to numpy array
    image = image.numpy()
    
    return image

def process_single_image(image_path, target_shape=(256, 256), detection_threshold=0.1):
    # Preprocess the image
    image = preprocess_image(image_path, target_shape)
    
    # Detect landmarks
    landmarks = detect_landmarks(image, detection_threshold)
    
    if landmarks is not None:
        # Convert landmarks to DataFrame
        list_name = [[bodypart.name + '_x', bodypart.name + '_y', bodypart.name + '_score'] for bodypart in BodyPart]
        
        header_name = []
        for columns_name in list_name:
            header_name += columns_name
        header_name = ['filename'] + header_name
        
        landmarks = landmarks.flatten().astype(str).tolist()
        landmarks = [''] + landmarks  # Empty string for filename
        landmarks_dict = {header_name[i]: [landmarks[i]] for i in range(len(header_name))}
        
        df = pd.DataFrame(landmarks_dict)
        
        return df
    else:
        return None

ModuleNotFoundError: No module named 'cv2'

In [25]:
image_path = 'test.jpg'
landmarks = process_single_image(image_path)
if landmarks is not None:
    print("Landmarks detected successfully:", landmarks)
else:
    print("No person detected or landmarks below threshold.")

NameError: name 'process_single_image' is not defined

In [26]:
landmarks.drop('filename', axis=1, inplace=True)

NameError: name 'landmarks' is not defined

In [27]:
landmarks.head()

NameError: name 'landmarks' is not defined

In [28]:
landmarks = landmarks.astype(float)

NameError: name 'landmarks' is not defined

In [29]:
X.shape

(1512, 51)

In [ ]:
test = preprocess_data(landmarks)

In [ ]:
arr = model.predict(test)
arr

In [ ]:
arr_flat = arr.flatten()

# Find the index of the maximum element
max_index = np.argmax(arr_flat)

# Get the value of the maximum element
max_value = arr_flat[max_index]

print("Index of the highest element:", max_index)
print("Value of the highest element:", max_value)

In [ ]:
# 0 = chair
# 1 = cobra
# 2 = dog
# 3 = no_pose
# 4 = shoulder_stand
# 5 = triangle
# 6 = tree
# 7 - warrior